In [30]:
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from numba import jit
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from tqdm.keras import TqdmCallback

In [2]:
df = pd.read_csv('export.csv')

In [3]:
df_next_event = df.copy()

#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

df_next_event['next_event'] = calculator_nb(df_next_event['case'].values, df_next_event['event'].values)

C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13436\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13436\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_n

In [4]:
df = df_next_event

In [5]:
goodshit = ['case','event','number_parcels','payment_actual0','area','cross_compliance','penalty_amount0','next_event']
df_reduced = df.copy()

for col in df.columns:
    if (goodshit.count(col) == 0):
        df_reduced = df_reduced.drop(col, 1)

C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_1343

C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_13436\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_1343

In [6]:
def normalize(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [11]:
def col_to_millis(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array[:] = [elem[:19] for elem in col_as_array]
    
    result = []
    for date in col_as_array:
        utctime = datetime.strptime(date, "%Y/%m/%d %H:%M:%S")
        result.append((utctime - datetime(1970, 1, 1)) // timedelta(milliseconds=1))
    
    return result

In [7]:
df_reduced = df_reduced.fillna(value="None")

In [9]:
df_reduced, validate, test = np.split(df_reduced.sample(frac=1), [int(.6*len(df_reduced)), int(.8*len(df_reduced))])

In [65]:
event = df_reduced['event'].to_numpy()
next_event = df_reduced['next_event'].to_numpy()

In [67]:
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()
event = event.reshape(-1, 1)
event_encoded = ordinal_encoder.fit_transform(event)
next_event_encoded = label_encoder.fit_transform(next_event)
next_event_encoded = next_event_encoded.reshape(-1, 1)

In [68]:
next_event_encoded

array([[20],
       [18],
       [20],
       ...,
       [ 9],
       [20],
       [ 9]])

In [56]:
number_parcels = normalize('number_parcels')
payment_actual0 = normalize('payment_actual0')
area = normalize('area')
cross_compliance = normalize('cross_compliance')
penalty_amount0 = normalize('penalty_amount0')

In [57]:
training = []
for i in range(len(event_encoded)):
    current = event_encoded[i]
    current = np.append(current,number_parcels[i])
    current = np.append(current,payment_actual0[i])
    current = np.append(current,area[i])
    current = np.append(current,cross_compliance[i])
    current = np.append(current,penalty_amount0[i])
    training.append(current)

In [17]:
labels = []
for i in range(len(next_event_encoded)):
    current = next_event_encoded[i]
    current = np.append(current,[0,0,0,0,0])
    labels.append(current)

In [58]:
labels = next_event_encoded

In [62]:
labels

array([[20],
       [18],
       [20],
       ...,
       [ 9],
       [20],
       [ 9]])

In [54]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(43, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [64]:
model.fit(np.array(training),np.array(labels),epochs=10,verbose=1)

Epoch 1/10
47143/47143 [==============================] - 76s 2ms/step - loss: 1.6577 - accuracy: 0.5044
Epoch 2/10
47143/47143 [==============================] - 77s 2ms/step - loss: 1.5367 - accuracy: 0.5292
Epoch 3/10
47143/47143 [==============================] - 75s 2ms/step - loss: 1.5022 - accuracy: 0.5394
Epoch 4/10
47143/47143 [==============================] - 77s 2ms/step - loss: 1.4845 - accuracy: 0.5465
Epoch 5/10
47143/47143 [==============================] - 76s 2ms/step - loss: 1.4715 - accuracy: 0.5508
Epoch 6/10
47143/47143 [==============================] - 76s 2ms/step - loss: 1.4618 - accuracy: 0.5536
Epoch 7/10
47143/47143 [==============================] - 77s 2ms/step - loss: 1.4547 - accuracy: 0.5554
Epoch 8/10
47143/47143 [==============================] - 76s 2ms/step - loss: 1.4492 - accuracy: 0.5571
Epoch 9/10
47143/47143 [==============================] - 76s 2ms/step - loss: 1.4442 - accuracy: 0.5583
Epoch 10/10
47143/47143 [==============================

In [74]:
def normalizetest(col_name):
    col_as_array = test[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [71]:
event_test = test['event'].to_numpy()
next_event_test = test['next_event'].to_numpy()

In [72]:
event_test = event_test.reshape(-1, 1)
event_encoded_test = ordinal_encoder.fit_transform(event_test)
next_event_encoded_test = label_encoder.fit_transform(next_event_test)
next_event_encoded_test = next_event_encoded_test.reshape(-1, 1)

In [75]:
number_parcels_test = normalizetest('number_parcels')
payment_actual0_test = normalizetest('payment_actual0')
area_test = normalizetest('area')
cross_compliance_test = normalizetest('cross_compliance')
penalty_amount0_test = normalizetest('penalty_amount0')

In [76]:
testing = []
for i in range(len(event_encoded_test)):
    current = event_encoded_test[i]
    current = np.append(current,number_parcels_test[i])
    current = np.append(current,payment_actual0_test[i])
    current = np.append(current,area_test[i])
    current = np.append(current,cross_compliance_test[i])
    current = np.append(current,penalty_amount0_test[i])
    testing.append(current)

In [78]:
labels_test = next_event_encoded_test

In [81]:
test_loss, test_acc = model.evaluate(np.array(testing), np.array(labels_test), verbose=1)
test_acc

15715/15715 [==============================] - 15s 941us/step - loss: 1.4320 - accuracy: 0.5610


0.5609620213508606